<a href="https://colab.research.google.com/github/CinthiaS/distill-roberta/blob/main/DistillRoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.9

In [2]:
import torch
from transformers import RobertaConfig, RobertaForMaskedLM, RobertaTokenizer, BertConfig, BertForMaskedLM, BertTokenizer

# Definir as configurações dos modelos
teacher_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
teacher_model = RobertaForMaskedLM.from_pretrained('roberta-base')
roberta_vocab_size = teacher_tokenizer.vocab_size

student_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
student_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
student_model.resize_token_embeddings(roberta_vocab_size)

new_bert_vocab_size = student_model.config.vocab_size
print("Novo tamanho do vocabulário do BERT:", new_bert_vocab_size)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Novo tamanho do vocabulário do BERT: 50265


In [3]:
texts = [
    "Eu amo esse filme!",
    "Que dia maravilhoso!",
    "Estou muito feliz com essa notícia.",
    "Esse livro é incrível!",
    "Não gostei desse restaurante.",
    "Que péssima experiência!",
    "Estou muito chateado com essa situação."
]
labels = [1, 1, 1, 1, 0, 0, 0] 

# Tokenizar os textos usando o tokenizador do modelo professor (Roberta)
input_ids = []
attention_masks = []
for text in texts:
    encoded_text = teacher_tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded_text['input_ids'])
    attention_masks.append(encoded_text['attention_mask'])

# Concatenar os tensores de entrada
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Definir os dados de treinamento
train_dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Definir o otimizador e a função de perda
optimizer = torch.optim.Adam(student_model.parameters(), lr=1e-4)
criterion = torch.nn.KLDivLoss()

# Definir a função de treinamento
def train(teacher_model, student_model, dataloader, optimizer, criterion, temperature):
    teacher_model.eval()
    student_model.train()
    for input_ids, attention_mask, labels in dataloader:
        optimizer.zero_grad()
        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits / temperature
        student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits / temperature
        loss = criterion(torch.nn.functional.log_softmax(student_logits, dim=-1), torch.nn.functional.softmax(teacher_logits, dim=-1))
        loss.backward()
        optimizer.step()

def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for input_ids, attention_mask, labels in dataloader:

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predictions = torch.max(logits, dim=1)

            loss = criterion(logits, labels)
            total_loss += loss.item()

            correct_predictions += torch.sum(predictions == labels).item()
            total_predictions += len(labels)

    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_predictions

    return average_loss, accuracy


# Treinar o modelo
num_epochs = 1
temperature = 2.0
for epoch in range(num_epochs):
    train(teacher_model, student_model, train_dataloader, optimizer, criterion, temperature)
    average_loss, accuracy = evaluate(student_model, train_dataloader, criterion)


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
